<a href="https://colab.research.google.com/github/Abhinash200301/Continuous-Time-Networks-for-Adaptive-Process-Control/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setups

In [ ]:
!pip install tensorflow==2.13.0
!pip install keras==2.13.1

In [ ]:
!pip install tensorflow_addons

In [ ]:
import glob
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow_addons.metrics import RSquare
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [ ]:
df= pd.read_csv("/content/PostDischarge_Data.csv")

In [ ]:
label=df["label"]
ct=[]
for i in np.unique(label):
  count=0
  for j in range(len(df)):
    if df["label"][j]== i:
      count=count+1
  ct.append(count)

In [ ]:
m=np.unique(label)

In [ ]:
l=0
for i in range(len(ct)):
  df1= df[l:l+ct[i]]
  with pd.ExcelWriter(m[i]+".xlsx") as writer:
    df1.to_excel(writer)
  l= ct[i]+l
  print(l)

349
529
730
764
1092
1304
1444
1480
1805
2122
2420
2729


In [ ]:
sequence_length = 5  # Length of the input sequence

In [ ]:
# Get the list of Excel files
a = glob.glob("*.xlsx")

# Lists to store inputs and outputs
train_input = []
train_output = []

# Loop through each file
for j in a:
    dfk = pd.read_excel(j)
    dfm = dfk.drop(columns=['label', 'cycle number', 'Unnamed: 0'])  # Drop unnecessary columns

    # Loop to create sequences based on the specified sequence length
    for i in range(len(dfk) - sequence_length):
        ink = []
        # Collecting the previous 'sequence_length' steps
        for n in range(sequence_length):
            ink.append(list(dfm[i + n:i + n + 1].values[0]))
        train_input.append(ink)

        # Predict the next value after the sequence
        train_output.append(list(dfm[i + sequence_length:i + sequence_length + 1].values[0])[0])

In [ ]:
# Convert lists to numpy arrays
train_input = np.array(train_input)
train_output = np.array(train_output)

In [ ]:
# Ensure train_output is reshaped to (samples, 1)
train_output = train_output.reshape(-1, 1)

In [ ]:
input_shape = (sequence_length, 121)

In [ ]:
def get_model(input_shape):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.LSTM(100, activation='relu', return_sequences=False))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(1, activation="linear"))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[RSquare()])
    return model

tf.keras.backend.clear_session()
model = get_model(input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               88800     
                                                                 
 dense (Dense)               (None, 64)                6464      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 95329 (372.38 KB)
Trainable params: 95329 (372.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train

In [ ]:
history = model.fit(train_input, train_output, epochs=150, validation_split=0.1, verbose=1)

Epoch 1/150
76/76 [==============================] - 4s 15ms/step - loss: 137.3932 - r_square: -3.7892 - val_loss: 1.7659 - val_r_square: 0.7366
Epoch 2/150
76/76 [==============================] - 1s 10ms/step - loss: 0.3032 - r_square: 0.9894 - val_loss: 0.9893 - val_r_square: 0.8525
Epoch 3/150
76/76 [==============================] - 1s 11ms/step - loss: 0.2262 - r_square: 0.9921 - val_loss: 0.9887 - val_r_square: 0.8525
Epoch 4/150
76/76 [==============================] - 1s 10ms/step - loss: 0.2296 - r_square: 0.9920 - val_loss: 0.9754 - val_r_square: 0.8545
Epoch 5/150
76/76 [==============================] - 1s 11ms/step - loss: 0.2247 - r_square: 0.9922 - val_loss: 0.9913 - val_r_square: 0.8522
Epoch 6/150
76/76 [==============================] - 1s 11ms/step - loss: 0.1832 - r_square: 0.9936 - val_loss: 1.2020 - val_r_square: 0.8207
Epoch 7/150
76/76 [==============================] - 1s 11ms/step - loss: 0.1967 - r_square: 0.9931 - val_loss: 0.9861 - val_r_square: 0.8529
Epo

In [ ]:
model.save("/content/content/my_best_model.hdf5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import keras
model1= keras.models.load_model("/content/content/my_best_model.hdf5")


In [ ]:
# List of battery files
a = glob.glob("*.xlsx")

# Loop through each battery file
for m in a:
    # Load the battery data
    df10 = pd.read_excel(m)
    df101 = df10.drop(columns=['label', 'cycle number', 'Unnamed: 0', "Capacity/mA.h"])

    # Initialize with the first 'sequence_length' capacity values for prediction
    true_capacity = list(df10["Capacity/mA.h"])  # True capacity values for R-squared calculation
    predicted_capacity = true_capacity[:sequence_length]  # First 'sequence_length' true values to start the prediction

    # Iterating through the battery data for predictions with adjustable sequence length
    for i in range(len(df101) - (sequence_length - 1)):
        # Create a sequence of 'sequence_length' steps (the model expects input of shape (sequence_length, 121))
        seq = [list(df101.iloc[i + n]) for n in range(sequence_length)]

        # Insert the true capacity values into the sequence at the appropriate places
        for n in range(sequence_length):
            seq[n].insert(0, predicted_capacity[i + n])  # Insert the corresponding capacity value

        # Convert the sequence into a numpy array and reshape it to (1, sequence_length, 121)
        seq = np.array(seq).reshape(1, sequence_length, 121)

        # Make the prediction using the Liquid Neural Network model
        predict = model1(tf.convert_to_tensor(seq, dtype=tf.float32))

        # Append the predicted capacity value
        predicted_capacity.append(predict.numpy()[0][0])

    # Since predicted_capacity has extra initial values, slice it to match true_capacity length
    predicted_capacity = predicted_capacity[:len(true_capacity)]

    # Compute R-squared value for the predictions
    r_squared = r2_score(true_capacity, predicted_capacity)
    print(f"R-squared for {m.split('.')[0]}: {r_squared}")

    # Save the predicted results for this battery
    results = pd.DataFrame({'True Capacity': true_capacity, 'Predicted Capacity': predicted_capacity})
    with pd.ExcelWriter("/content/content/" + m.split('.')[0] + "_results_with_prediction.xlsx") as writer:
        results.to_excel(writer, index=False)

    # Plot the true and predicted capacity values for this battery
    plt.figure()  # Create a new figure for each battery
    plt.plot(true_capacity, label='True Capacity', color='blue')
    plt.plot(predicted_capacity, label='Predicted Capacity', color='red')
    plt.title(f"True vs Predicted Capacity over Cycles for {m.split('.')[0]}")
    plt.xlabel("Cycle Number")
    plt.ylabel("Capacity (mA.h)")
    plt.legend()
    plt.savefig(f"{m.split('.')[0]}_Capacity_True_vs_Predicted.png")  # Save the plot as a PNG file
    plt.close()  # Close the plot to avoid overlap with the next one



R-squared for 25C03: 0.9220701494344158
R-squared for 45C01: 0.9181573912539158
R-squared for 25C01: 0.8116213442188918
R-squared for 25C04: 0.8846314622250739
R-squared for 25C07: 0.6259620391553309
R-squared for 35C01: 0.38693299341134924
R-squared for 25C05: 0.7871019924623952
R-squared for 25C02: -2.249969310740224
R-squared for 25C08: 0.5060268132785087
R-squared for 25C06: -3.0258410357614487
R-squared for 35C02: 0.5162240518573302
R-squared for 45C02: 0.8385637939240064
